In [1]:
%reload_kedro

2020-04-10 15:20:38,742 - root - INFO - ** Kedro project Dynamic Topic Modeling
2020-04-10 15:20:38,742 - root - INFO - Defined global variable `context` and `catalog`
Traceback (most recent call last):
  File "/home/ridouane/anaconda3/envs/dynamic-topic-modeling/lib/python3.6/site-packages/kedro/cli/cli.py", line 589, in load_entry_points
    entry_point_commands.append(entry_point.load())
  File "/home/ridouane/anaconda3/envs/dynamic-topic-modeling/lib/python3.6/site-packages/pkg_resources/__init__.py", line 2449, in load
    self.require(*args, **kwargs)
  File "/home/ridouane/anaconda3/envs/dynamic-topic-modeling/lib/python3.6/site-packages/pkg_resources/__init__.py", line 2472, in require
    items = working_set.resolve(reqs, env, installer, extras=self.extras)
  File "/home/ridouane/anaconda3/envs/dynamic-topic-modeling/lib/python3.6/site-packages/pkg_resources/__init__.py", line 792, in resolve
    raise VersionConflict(dist, req).with_context(dependent_req)
pkg_resources.Versio

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
dataset = catalog.load('raw_dataset')
train_docs = catalog.load('train_docs')
vocab = catalog.load('dictionary')

2020-04-10 15:20:39,494 - kedro.io.data_catalog - INFO - Loading data from `raw_dataset` (CSVLocalDataSet)...
2020-04-10 15:20:42,189 - kedro.io.data_catalog - INFO - Loading data from `train_docs` (PickleLocalDataSet)...
2020-04-10 15:20:43,701 - kedro.io.data_catalog - INFO - Loading data from `dictionary` (DictionaryDataSet)...
2020-04-10 15:20:43,702 - gensim.utils - INFO - loading Dictionary object from data/05_model_input/dictionary.dict
2020-04-10 15:20:43,716 - gensim.utils - INFO - loaded data/05_model_input/dictionary.dict


## Pre-process docs

In [4]:
def split_by_sentence(docs):
    tmp = []
    for i, doc in enumerate(docs):
        splitted_doc = doc.split('.\n')
        for sd in splitted_doc:
            sentences = sd.split('. ')
            for s in sentences:
                tmp.append(s)
    return tmp

def lowerize(docs):
    # Convert to lowercase.
    for idx in range(len(docs)):
        docs[idx] = str(docs[idx]).lower()
    return docs

def tokenize(docs):
    # Split into words.
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = tokenizer.tokenize(docs[idx])
    return docs

def remove_stop_words(docs):
    stop_words = set(stopwords.words('english'))
    for idx in range(len(docs)):
        docs[idx] = [w for w in docs[idx] if not w in stop_words]
    return docs

def remove_numbers(docs):
    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isnumeric()] for doc in docs]
    return docs

def remove_word_with_length(docs, length=1):
    # Remove words that are only (length=1) character.
    docs = [[token for token in doc if len(token) > length] for doc in docs]
    return docs

def lemmatize(docs):
    # Lemmatize the documents
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
    return docs

def add_bigram(docs, min_bigram_count=20):
    # Add bigrams and trigrams to docs (only ones that appear 20 times or more).
    bigram = Phrases(docs, min_count=min_bigram_count)
    for idx in range(len(docs)):
        for token in bigram[docs[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                docs[idx].append(token)
    return docs

def remove_vocab(docs, vocab):
    docs = np.array([[w for w in doc if w in vocab] for doc in docs])
    return docs

In [5]:
min_bigram_count = 20
length = 1
no_below = 100
no_above = 0.70

docs = dataset['text'].values

print('\nSplitting by sentence...')
docs = split_by_sentence(docs)

print('\nLowerizing...')
docs = lowerize(docs)

print('\nTokenizing...')
docs = tokenize(docs)

#print('\nAdding bigrams...')
#docs = add_bigram(docs, min_bigram_count=min_bigram_count)
    
print('\nRemoving stop words...')
docs = remove_stop_words(docs)

print('\nRemoving unique numbers (not words that contain numbers)...')
docs = remove_numbers(docs)

print('\nRemoving words that contain only one character...')
docs = remove_word_with_length(docs, length=length)

print('\nLemmatizing...')
docs = lemmatize(docs)

vocab = Dictionary(docs)
vocab.filter_extremes(no_below=no_below, no_above=no_above)

docs = remove_vocab(docs, list(vocab.token2id))

print('Number of sentences:', len(docs))
print('Number of unique words:', len(vocab))


Splitting by sentence...

Lowerizing...

Tokenizing...

Removing stop words...

Removing unique numbers (not words that contain numbers)...

Removing words that contain only one character...

Lemmatizing...
2020-04-10 15:21:36,242 - gensim.corpora.dictionary - INFO - adding document #0 to Dictionary(0 unique tokens: [])
2020-04-10 15:21:36,430 - gensim.corpora.dictionary - INFO - adding document #10000 to Dictionary(9529 unique tokens: ['ambassador', 'assembly', 'congratulation', 'delegation', 'election']...)
2020-04-10 15:21:36,622 - gensim.corpora.dictionary - INFO - adding document #20000 to Dictionary(12395 unique tokens: ['ambassador', 'assembly', 'congratulation', 'delegation', 'election']...)
2020-04-10 15:21:36,820 - gensim.corpora.dictionary - INFO - adding document #30000 to Dictionary(15222 unique tokens: ['ambassador', 'assembly', 'congratulation', 'delegation', 'election']...)
2020-04-10 15:21:37,002 - gensim.corpora.dictionary - INFO - adding document #40000 to Dictionar

2020-04-10 15:21:44,592 - gensim.corpora.dictionary - INFO - adding document #420000 to Dictionary(36535 unique tokens: ['ambassador', 'assembly', 'congratulation', 'delegation', 'election']...)
2020-04-10 15:21:44,805 - gensim.corpora.dictionary - INFO - adding document #430000 to Dictionary(36785 unique tokens: ['ambassador', 'assembly', 'congratulation', 'delegation', 'election']...)
2020-04-10 15:21:45,029 - gensim.corpora.dictionary - INFO - adding document #440000 to Dictionary(37465 unique tokens: ['ambassador', 'assembly', 'congratulation', 'delegation', 'election']...)
2020-04-10 15:21:45,253 - gensim.corpora.dictionary - INFO - adding document #450000 to Dictionary(38154 unique tokens: ['ambassador', 'assembly', 'congratulation', 'delegation', 'election']...)
2020-04-10 15:21:45,495 - gensim.corpora.dictionary - INFO - adding document #460000 to Dictionary(38527 unique tokens: ['ambassador', 'assembly', 'congratulation', 'delegation', 'election']...)
2020-04-10 15:21:45,695 -

Number of sentences: 798974
Number of unique words: 6935


## Train doc embeddings (gensim doc2vec)

In [6]:
def read_corpus(docs):
    for i, text in enumerate(docs):
        yield TaggedDocument(text, [i])

In [7]:
corpus = list(read_corpus(docs))

In [8]:
size = 300
window = 5
min_count = 1
workers = 8
epochs = 50

model = Doc2Vec(vector_size=size, window=window, min_count=min_count, workers=workers, epochs=epochs)

In [9]:
model.build_vocab(corpus)

2020-04-10 15:25:33,213 - gensim.models.doc2vec - INFO - collecting all words and their counts
2020-04-10 15:25:33,216 - gensim.models.doc2vec - INFO - PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-04-10 15:25:33,251 - gensim.models.doc2vec - INFO - PROGRESS: at example #10000, processed 137391 words (4334795/s), 6041 word types, 10000 tags
2020-04-10 15:25:33,282 - gensim.models.doc2vec - INFO - PROGRESS: at example #20000, processed 272256 words (4412666/s), 6527 word types, 20000 tags
2020-04-10 15:25:33,317 - gensim.models.doc2vec - INFO - PROGRESS: at example #30000, processed 416218 words (4270928/s), 6719 word types, 30000 tags
2020-04-10 15:25:33,346 - gensim.models.doc2vec - INFO - PROGRESS: at example #40000, processed 540054 words (4371889/s), 6814 word types, 40000 tags
2020-04-10 15:25:33,375 - gensim.models.doc2vec - INFO - PROGRESS: at example #50000, processed 674162 words (4746281/s), 6846 word types, 50000 tags
2020-04-10 15:25:33,407 - g

2020-04-10 15:25:34,878 - gensim.models.doc2vec - INFO - PROGRESS: at example #540000, processed 7357713 words (4428320/s), 6930 word types, 540000 tags
2020-04-10 15:25:34,908 - gensim.models.doc2vec - INFO - PROGRESS: at example #550000, processed 7480225 words (4221973/s), 6930 word types, 550000 tags
2020-04-10 15:25:34,939 - gensim.models.doc2vec - INFO - PROGRESS: at example #560000, processed 7616567 words (4487685/s), 6930 word types, 560000 tags
2020-04-10 15:25:34,971 - gensim.models.doc2vec - INFO - PROGRESS: at example #570000, processed 7760896 words (4583955/s), 6930 word types, 570000 tags
2020-04-10 15:25:35,004 - gensim.models.doc2vec - INFO - PROGRESS: at example #580000, processed 7892026 words (4114987/s), 6930 word types, 580000 tags
2020-04-10 15:25:35,033 - gensim.models.doc2vec - INFO - PROGRESS: at example #590000, processed 8014610 words (4349580/s), 6934 word types, 590000 tags
2020-04-10 15:25:35,061 - gensim.models.doc2vec - INFO - PROGRESS: at example #600

In [10]:
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

2020-04-10 15:27:44,701 - gensim.models.base_any2vec - INFO - training model with 8 workers on 6935 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2020-04-10 15:27:45,851 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 0.80% examples, 79145 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:27:47,167 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 2.29% examples, 102529 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:27:48,354 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 3.69% examples, 112997 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:27:49,756 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 5.29% examples, 113788 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:27:50,906 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 6.75% examples, 118547 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:27:52,183 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 8.16% examples, 119721 words

2020-04-10 15:28:47,715 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 72.30% examples, 125579 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:28:48,848 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 73.43% examples, 125101 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:28:49,869 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 74.76% examples, 125299 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:28:50,888 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 75.89% examples, 125188 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:28:52,024 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 77.18% examples, 125306 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:28:53,104 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 78.48% examples, 125522 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:28:54,312 - gensim.models.base_any2vec - INFO - EPOCH 1 - PROGRESS: at 79.88% examples, 125652 words/s, in_qsize 15, out_qsize 0

2020-04-10 15:29:42,766 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 36.51% examples, 126362 words/s, in_qsize 16, out_qsize 0
2020-04-10 15:29:43,894 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 37.48% examples, 125090 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:29:45,010 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 38.95% examples, 125452 words/s, in_qsize 16, out_qsize 0
2020-04-10 15:29:46,152 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 40.35% examples, 125938 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:29:47,261 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 41.69% examples, 126491 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:29:48,297 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 42.89% examples, 126742 words/s, in_qsize 16, out_qsize 0
2020-04-10 15:29:49,434 - gensim.models.base_any2vec - INFO - EPOCH 2 - PROGRESS: at 44.10% examples, 126641 words/s, in_qsize 14, out_qsize 1

2020-04-10 15:30:36,914 - gensim.models.base_any2vec - INFO - worker thread finished; awaiting finish of 0 more threads
2020-04-10 15:30:36,915 - gensim.models.base_any2vec - INFO - EPOCH - 2 : training on 10849021 raw words (10916325 effective words) took 85.7s, 127352 effective words/s
2020-04-10 15:30:38,042 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 0.80% examples, 80737 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:30:39,349 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 2.29% examples, 103773 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:30:40,553 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 3.69% examples, 113474 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:30:41,964 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 5.29% examples, 113966 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:30:43,099 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 6.75% examples, 118974 words/s, in_qsize 15, out_qsize 0
20

2020-04-10 15:31:38,278 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 71.02% examples, 126654 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:31:39,287 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 72.10% examples, 126534 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:31:40,297 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 73.33% examples, 126438 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:31:41,315 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 74.49% examples, 126313 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:31:42,334 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 75.69% examples, 126337 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:31:43,361 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 77.03% examples, 126647 words/s, in_qsize 15, out_qsize 0
2020-04-10 15:31:44,393 - gensim.models.base_any2vec - INFO - EPOCH 3 - PROGRESS: at 78.04% examples, 126486 words/s, in_qsize 15, out_qsize 0

...

2020-04-10 16:35:13,558 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 13.24% examples, 123912 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:35:14,630 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 14.45% examples, 124507 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:35:15,706 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 15.62% examples, 123542 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:35:16,716 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 17.01% examples, 125297 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:35:17,723 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 18.04% examples, 124903 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:35:18,815 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 19.29% examples, 125127 words/s, in_qsize 16, out_qsize 0
2020-04-10 16:35:19,823 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 20.51% examples, 125361 words/s, in_qsize 15, out_

2020-04-10 16:36:14,493 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 84.28% examples, 127330 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:36:15,570 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 85.70% examples, 127511 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:36:16,589 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 86.80% examples, 127389 words/s, in_qsize 16, out_qsize 0
2020-04-10 16:36:17,729 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 88.08% examples, 127326 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:36:18,766 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 89.30% examples, 127435 words/s, in_qsize 16, out_qsize 0
2020-04-10 16:36:19,809 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 90.58% examples, 127534 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:36:20,833 - gensim.models.base_any2vec - INFO - EPOCH 48 - PROGRESS: at 91.70% examples, 127407 words/s, in_qsize 16, out_

2020-04-10 16:37:09,887 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 49.29% examples, 128161 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:37:10,911 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 50.42% examples, 127916 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:37:11,991 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 51.79% examples, 127988 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:37:13,021 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 53.11% examples, 128401 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:37:14,024 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 54.07% examples, 128006 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:37:15,029 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 55.33% examples, 128264 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:37:16,040 - gensim.models.base_any2vec - INFO - EPOCH 49 - PROGRESS: at 56.54% examples, 128490 words/s, in_qsize 16, out_

2020-04-10 16:38:04,414 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 13.24% examples, 123257 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:38:05,489 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 14.45% examples, 123863 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:38:06,550 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 15.62% examples, 123096 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:38:07,554 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 17.01% examples, 124911 words/s, in_qsize 16, out_qsize 0
2020-04-10 16:38:08,554 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 18.14% examples, 125223 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:38:09,639 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 19.29% examples, 124865 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:38:10,639 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 20.51% examples, 125172 words/s, in_qsize 16, out_

2020-04-10 16:39:06,248 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 85.70% examples, 127705 words/s, in_qsize 16, out_qsize 0
2020-04-10 16:39:07,277 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 86.89% examples, 127698 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:39:08,427 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 88.08% examples, 127482 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:39:09,442 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 89.30% examples, 127626 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:39:10,479 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 90.58% examples, 127737 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:39:11,521 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 91.70% examples, 127577 words/s, in_qsize 15, out_qsize 0
2020-04-10 16:39:12,558 - gensim.models.base_any2vec - INFO - EPOCH 50 - PROGRESS: at 92.94% examples, 127674 words/s, in_qsize 15, out_

In [12]:
model.save('./data/04_features/doc2vec_embeddings.bin')

2020-04-10 16:39:53,218 - gensim.utils - INFO - saving Doc2Vec object under ./data/04_features/doc2vec_embeddings.bin, separately None
2020-04-10 16:39:53,220 - gensim.utils - INFO - storing np array 'vectors_docs' to ./data/04_features/doc2vec_embeddings.bin.docvecs.vectors_docs.npy
2020-04-10 16:40:00,382 - gensim.utils - INFO - saved ./data/04_features/doc2vec_embeddings.bin


## Get embeddings

In [13]:
from sklearn import preprocessing
doc_embeddings = np.array([model.infer_vector(doc) for doc in train_docs])
doc_embeddings_norm = preprocessing.normalize(doc_embeddings)

In [14]:
import pickle
with open('./data/04_features/doc_embeddings.pkl', 'wb') as handle:
    pickle.dump(doc_embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./data/04_features/doc_embeddings_norm.pkl', 'wb') as handle:
    pickle.dump(doc_embeddings_norm, handle, protocol=pickle.HIGHEST_PROTOCOL)